In [337]:
import pandas as pd
import random
import datetime as dt

In [372]:
mulai_pengajaran = dt.datetime(year=2020, month=9, day=21) #senin
df = pd.read_csv("tbl0.csv")
df.index.name = "index"

# Pendaftaran tutup setelah total jam 60|61|62

In [373]:
#pendaftaran ditutup saat total jam pengajar mencapai 60|61|62
#maka, sort database dengan waktu, ambil data dari awal hingga jam nya 60|61|62 sort
df = df.sort_values("waktu_dftr")
df = df.reset_index()
df["cumsum"] = df["jam"].cumsum()
limit_hours = [60, 61, 62]
from_limits = [df.where((df["cumsum"]==i)).dropna() for i in limit_hours]

print(df)


    index  Unnamed: 0     nama  waktu_dftr  jam  senin  selasa  rabu  kamis  \
0      86          86  right86  2020-09-12    2      1       1     1      1   
1      62          62  right62  2020-09-12    1      1       1     1      1   
2      15          15  right15  2020-09-12    3      0       1     1      1   
3      81          81  right81  2020-09-12    3      1       0     0      0   
4      59          59  right59  2020-09-12    2      1       0     1      1   
..    ...         ...      ...         ...  ...    ...     ...   ...    ...   
95     63          63  right63  2020-09-21    2      1       1     0      1   
96     78          78  right78  2020-09-21    2      0       0     0      1   
97     70          70  right70  2020-09-21    1      1       0     1      1   
98     56          56  right56  2020-09-21    2      1       1     1      1   
99     49          49  right49  2020-09-21    1      1       0     1      0   

    jumat  sabtu  cumsum  
0       1      1       2

In [374]:
print(df.iloc[21])

index                 48
Unnamed: 0            48
nama             right48
waktu_dftr    2020-09-14
jam                    2
senin                  1
selasa                 1
rabu                   1
kamis                  1
jumat                  1
sabtu                  1
cumsum                51
Name: 21, dtype: object


In [375]:
#hapus dari from_limits yang tidak ada datanya
from_limits = [i for i in from_limits if len(i["nama"])!=0]
print(from_limits)


[    index  Unnamed: 0    nama  waktu_dftr  jam  senin  selasa  rabu  kamis  \
27    9.0         9.0  right9  2020-09-14  3.0    0.0     0.0   0.0    1.0   

    jumat  sabtu  cumsum  
27    1.0    0.0    62.0  ]


In [376]:
#ambil yang terbanyak
data_limit = from_limits[-1]
print(data_limit)


    index  Unnamed: 0    nama  waktu_dftr  jam  senin  selasa  rabu  kamis  \
27    9.0         9.0  right9  2020-09-14  3.0    0.0     0.0   0.0    1.0   

    jumat  sabtu  cumsum  
27    1.0    0.0    62.0  


In [377]:
#dapatkan indexnya
index_limit = int(data_limit.index.tolist()[0])
print(index_limit)

27


In [378]:
#ambil data dengan slicing 
new_df = df.iloc[0:index_limit+1]
del new_df["index"]
del new_df["Unnamed: 0"]
print(new_df)

new_df.to_csv("temp0.csv")

       nama  waktu_dftr  jam  senin  selasa  rabu  kamis  jumat  sabtu  cumsum
0   right86  2020-09-12    2      1       1     1      1      1      1       2
1   right62  2020-09-12    1      1       1     1      1      1      1       3
2   right15  2020-09-12    3      0       1     1      1      1      0       6
3   right81  2020-09-12    3      1       0     0      0      0      0       9
4   right59  2020-09-12    2      1       0     1      1      1      1      11
5   right52  2020-09-12    3      0       0     1      0      0      1      14
6   right91  2020-09-12    3      1       1     1      1      1      1      17
7   right69  2020-09-12    3      0       0     0      0      1      0      20
8   right46  2020-09-12    2      1       1     0      1      1      1      22
9   right94  2020-09-12    2      0       1     0      0      0      0      24
10  right95  2020-09-12    3      1       1     1      0      0      1      27
11  right96  2020-09-12    3      1       1     1   

# Sedapat mungkin 48 jam dan Sebanyak-banyak mungkin instrukturnya

In [379]:
all_days = "senin selasa rabu kamis jumat sabtu".split(" ")

df1 = new_df.copy()

#tambahkan kolom jumlah hari mengajar
df1["day_counts"] = df1[all_days].sum(axis=1)
#tambahkan kolom total_hours
df1["total_hours"] = df1["day_counts"]*df1["jam"]
print(df1)

       nama  waktu_dftr  jam  senin  selasa  rabu  kamis  jumat  sabtu  \
0   right86  2020-09-12    2      1       1     1      1      1      1   
1   right62  2020-09-12    1      1       1     1      1      1      1   
2   right15  2020-09-12    3      0       1     1      1      1      0   
3   right81  2020-09-12    3      1       0     0      0      0      0   
4   right59  2020-09-12    2      1       0     1      1      1      1   
5   right52  2020-09-12    3      0       0     1      0      0      1   
6   right91  2020-09-12    3      1       1     1      1      1      1   
7   right69  2020-09-12    3      0       0     0      0      1      0   
8   right46  2020-09-12    2      1       1     0      1      1      1   
9   right94  2020-09-12    2      0       1     0      0      0      0   
10  right95  2020-09-12    3      1       1     1      0      0      1   
11  right96  2020-09-12    3      1       1     1      1      1      1   
12  right41  2020-09-12    3      1   

In [380]:
df2 = df1.sort_values(["total_hours", "day_counts"])
df2.reset_index(inplace=True)
del df2["cumsum"]
del df2["index"]
df2


,nama,waktu_dftr,jam,senin,selasa,rabu,kamis,jumat,sabtu,day_counts,total_hours
0,right25,2020-09-13,1,0,0,0,0,1,0,1,1
1,right94,2020-09-12,2,0,1,0,0,0,0,1,2
2,right81,2020-09-12,3,1,0,0,0,0,0,1,3
3,right69,2020-09-12,3,0,0,0,0,1,0,1,3
4,right27,2020-09-13,2,1,0,1,0,0,0,2,4
5,right30,2020-09-14,1,1,1,1,0,0,1,4,4
6,right99,2020-09-14,1,1,1,0,1,1,0,4,4
7,right52,2020-09-12,3,0,0,1,0,0,1,2,6
8,right38,2020-09-14,3,0,1,0,1,0,0,2,6
9,right9,2020-09-14,3,0,0,0,1,1,0,2,6


In [381]:
#peminat hari2 dengan di sort dari yang paling sedikit peminatnya
dict_peminat_hari = dict()
sum_peminat_hari = [sum(df2[i]) for i in all_days]
index = 0
for day in all_days:
    dict_peminat_hari[day] = sum_peminat_hari[index]
    index+=1
sorted_hari_peminat = sorted(dict_peminat_hari, key=dict_peminat_hari.get)
# print(sorted_hari_peminat)


hari_nama = dict()
for hari in sorted_hari_peminat:
    yg_bs_hari_x = df2[df2[hari]==1]
    yg_bs_hari_x["cumsum_hari_x"] = (yg_bs_hari_x[hari]*yg_bs_hari_x["jam"]).cumsum()
    approach_8 = yg_bs_hari_x.where((yg_bs_hari_x["cumsum_hari_x"]<=8)).dropna()
#     print(approach_8)
    hari_nama[hari] = [list(approach_8["nama"]), list(approach_8["jam"])]
    
print(hari_nama)

{'rabu': [['right27', 'right30', 'right52', 'right93'], [2.0, 1.0, 3.0, 2.0]], 'sabtu': [['right30', 'right52', 'right62', 'right60', 'right7'], [1.0, 3.0, 1.0, 1.0, 2.0]], 'selasa': [['right94', 'right30', 'right99', 'right38'], [2.0, 1.0, 1.0, 3.0]], 'kamis': [['right99', 'right38', 'right9'], [1.0, 3.0, 3.0]], 'jumat': [['right25', 'right69', 'right99', 'right9'], [1.0, 3.0, 1.0, 3.0]], 'senin': [['right81', 'right27', 'right30', 'right99'], [3.0, 2.0, 1.0, 1.0]]}


/home/kevin/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


In [382]:
#konfirmasi hasil -> harusnya tiap hari ada approach 8
dict_jamperhari = dict()
for hari in hari_nama:
    jam = 0
    nama_list = hari_nama[hari][0]
    for nama in nama_list:
        jam += (df2[df2["nama"]==nama]["jam"] * df2[df2["nama"]==nama][hari]).tolist()[0]
    dict_jamperhari[hari] = jam

total_jam = sum(list(dict_jamperhari.values()))
print(total_jam)
print(dict_jamperhari)

45
{'rabu': 8, 'sabtu': 8, 'selasa': 7, 'kamis': 7, 'jumat': 8, 'senin': 7}


# Finishing

In [383]:
#yang akan kerja minggu depan
yg_kerja_mingdep = []
for hari in hari_nama:
    nama_list = hari_nama[hari][0]
    yg_kerja_mingdep += nama_list
yg_kerja_mingdep = list(set(yg_kerja_mingdep))
print(yg_kerja_mingdep)

['right81', 'right69', 'right52', 'right93', 'right62', 'right94', 'right99', 'right7', 'right9', 'right27', 'right60', 'right30', 'right25', 'right38']


In [402]:
#yang ditunda
all_pendaftar = list(new_df["nama"])
list_tertunda = []
for n0 in all_pendaftar:
    if not (n0 in yg_kerja_mingdep):
        tertunda = new_df[new_df["nama"]==n0]
        list_tertunda.append([tertunda.nama.tolist()[0], tertunda.jam.tolist()[0]])
        
print(list_tertunda)  

[['right86', 2], ['right15', 3], ['right59', 2], ['right91', 3], ['right46', 2], ['right95', 3], ['right96', 3], ['right41', 3], ['right45', 2], ['right80', 2], ['right44', 3], ['right48', 2], ['right73', 2], ['right42', 2]]


In [410]:
#hasilkan output yang di inginkan
import json
# instruktur_terpilih = json.dumps(hari_nama)
# instruktur_tertunda = json.dumps(list_tertunda)
jadwal = ["{}.00 - {}.00".format(i, i+1) for i in range(8, 16)]
tbl_out = {"Jadwal": jadwal}

for day in all_days:
    tbl_out[day] = []


for hari in hari_nama:
    for i in range(len(hari_nama[hari][0])):
        i = int(i)
        for a in range(int(hari_nama[hari][1][i])):
            a = int(a)
            tbl_out[hari].append(hari_nama[hari][0][i])

            
for hari in all_days:
    tbl_out[hari] += [0]*(len(jadwal)-len(tbl_out[hari]))
    
for key in tbl_out:
    print(key, len(tbl_out[key]))
    
df_out = pd.DataFrame(tbl_out)
json_struct = {"hasil": []}
json_struct["hasil"] = json.loads(df_out.to_json(orient="records"))
with open("hasil.json", "w+") as f:
    json.dump(json_struct, f)

print("\n\nhasil akhir: ")
print(df_out)

# index = 0
# for key in tbl_out:
#     json_struct["hasil"].append()

Jadwal 8
senin 8
selasa 8
rabu 8
kamis 8
jumat 8
sabtu 8
          Jadwal    senin   selasa     rabu    kamis    jumat    sabtu
0    8.00 - 9.00  right81  right94  right27  right99  right25  right30
1   9.00 - 10.00  right81  right94  right27  right38  right69  right52
2  10.00 - 11.00  right81  right30  right30  right38  right69  right52
3  11.00 - 12.00  right27  right99  right52  right38  right69  right52
4  12.00 - 13.00  right27  right38  right52   right9  right99  right62
5  13.00 - 14.00  right30  right38  right52   right9   right9  right60
6  14.00 - 15.00  right99  right38  right93   right9   right9   right7
7  15.00 - 16.00        0        0  right93        0   right9   right7
